In [1]:
import os
import requests
import sqlite3
import pandas as pd

In [2]:
# directory of the raw data files
_data_root = 'data/data_mod/'

# location of the pipeline metadata store
_pipeline_root = './pipeline/'

In [3]:
import pandas as pd
import tensorflow as tf
import tfx
from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.v1.components import ImportSchemaGen
import tensorflow_data_validation as tfdv


from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

2024-03-04 04:32:11.830148: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 04:32:12.580128: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-04 04:32:12.580224: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [4]:
# Configuracion de contexto interactivo
context = InteractiveContext(pipeline_root=_pipeline_root)

### Examplex gen

In [5]:
# Instantiate ExampleGen with the input CSV dataset
#example_gen = CsvExampleGen(input_base=_data_root)

example_gen = CsvExampleGen(input_base=_data_root)

In [6]:
# Execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 75
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Estadisticas

In [7]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 76
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

### Inferir el esquema

In [9]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 77
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### Curando el esquema

In [11]:
from tensorflow_metadata.proto.v0 import schema_pb2

try:
    # Get the schema uri
    schema_uri = schema_gen.outputs['schema']._artifacts[0].uri
# for grading since context.run() does not work outside the notebook
except IndexError:
    print("context.run() was no-op")
    schema_path = './pipeline/SchemaGen/schema'
    dir_id = os.listdir(schema_path)[0]
    schema_uri = f'{schema_path}/{dir_id}'
    
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))



In [12]:
# Modificando Hillshade_9am de 0 a 255
tfdv.set_domain(schema, "Hillshade_9am", schema_pb2.IntDomain(name = "Hillshade_9am", min = 0 , max = 255 ))

# Modificando Hillshade_Noon de 0 a 255
tfdv.set_domain(schema, "Hillshade_Noon", schema_pb2.IntDomain(name = "Hillshade_Noon", min = 0 , max = 255 ))

# Modificando Slope de 0 a 90
tfdv.set_domain(schema, "Slope", schema_pb2.IntDomain(name = "Slope", min = 0 , max = 90 ))

# Modificando Cover_Type de 0 a 6
#tfdv.set_domain(schema, "Cover_Type", schema_pb2.StringDomain(name = "Cover_Type", value=['0','1', '2', '3','4','5','6'] ))


tfdv.set_domain(schema,'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))


In [13]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### Entorno de Esquemas

In [14]:
serv_data_df = pd.read_csv("data/data_mod/datos_finales.csv")
serv_data_df = serv_data_df.sample(frac = 0.30, random_state=42) 
serv_data_df = serv_data_df.drop(columns=["Cover_Type"])

# Directorio para guardar los datos de servicio
SERVING_DIR = 'data/serving'
os.makedirs(SERVING_DIR, exist_ok=True)

# Guardar los datos de servicio en un archivo CSV
SERVING_DATA_CSV = os.path.join(SERVING_DIR, 'serving_dataset.csv')
serv_data_df.to_csv(SERVING_DATA_CSV, index=False)

# Definir opciones de estadísticas
stats_options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)

# Calcular estadísticas del conjunto de datos de servicio
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA_CSV, stats_options=stats_options)

# Detectar anomalías en el conjunto de datos de servicio
anomalies = tfdv.validate_statistics(serving_stats, schema=schema)

# Mostrar las anomalías detectadas
tfdv.display_anomalies(anomalies)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


In [15]:
schema.default_environment.append('TRAINING')

### START CODE HERE ###
# Hint: Create another default schema environment with name SERVING (pass in a string)
schema.default_environment.append('SERVING')

# Remove Cover_Type feature from SERVING using TFDV
# Hint: Pass in the strings with the name of the feature and environment 
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')
### END CODE HERE ###

In [16]:
# Validate the serving dataset statistics in the `SERVING` environment
anomalies = tfdv.validate_statistics(serving_stats, schema=schema, environment='SERVING')

# Display the anomalies detected
tfdv.display_anomalies(anomalies)

In [17]:
# Declare the path to the updated schema directory
UPDATED_SCHEMA_DIR = './pipeline/updated_schema'

# Create the said directory
!mkdir -p {UPDATED_SCHEMA_DIR}

# Declare the path to the schema file
schema_file = os.path.join(UPDATED_SCHEMA_DIR, 'schema.pbtxt')

# Save the curated schema to the said file
tfdv.write_schema_text(schema, schema_file)


In [18]:
# Load the schema from the directory we just created
new_schema = tfdv.load_schema_text(schema_file)

# Display the schema. Check that the Domain column still contains the ranges.
tfdv.display_schema(schema=new_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### Nuevas Estadisticas

In [19]:
# Define la ruta al archivo de esquema
imported_schema = ImportSchemaGen(schema_file=schema_file)

In [20]:
# Run the component
context.run(imported_schema, enable_cache=False)

### END CODE HERE ###

context.show(imported_schema.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [21]:
# Use StatisticsGen to compute the statistics using the curated schema
statistics_gen_updated = StatisticsGen(examples=example_gen.outputs['examples'], 
                                       stats_options=tfdv.StatsOptions(infer_type_from_schema=True),
                                      schema=imported_schema.outputs['schema'])
    

# Run the component
context.run(statistics_gen_updated)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 79
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [22]:
context.show(statistics_gen_updated.outputs['statistics'])

### Comprobar anomalias

In [23]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=imported_schema.outputs['schema'])
    
# Run the component.
context.run(example_validator)


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 80
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [24]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

### Ingenierıa de caracterısticas

In [25]:
_cover_constants_module_file = 'cover_constants.py'

In [26]:
%%writefile {_cover_constants_module_file}

SCALE_MINMAX_FEATURE_KEYS = [
        "Horizontal_Distance_To_Hydrology",
        "Vertical_Distance_To_Hydrology",
    ]

SCALE_01_FEATURE_KEYS = [
        "Hillshade_9am",
        "Hillshade_Noon",
        "Horizontal_Distance_To_Fire_Points",
    ]

SCALE_Z_FEATURE_KEYS = [
        "Elevation",
        "Slope",
        "Horizontal_Distance_To_Roadways",
    ]

VOCAB_FEATURE_KEYS = ["Wilderness_Area"]

HASH_STRING_FEATURE_KEYS = ["Soil_Type"]

LABEL_KEY = "Cover_Type"

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Overwriting cover_constants.py


In [27]:
# Set the transform module filename
_cover_transform_module_file = 'cover_transform.py'

In [28]:
%%writefile {_cover_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import cover_constants

_SCALE_MINMAX_FEATURE_KEYS = cover_constants.SCALE_MINMAX_FEATURE_KEYS
_SCALE_01_FEATURE_KEYS = cover_constants.SCALE_01_FEATURE_KEYS
_SCALE_Z_FEATURE_KEYS = cover_constants.SCALE_Z_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = cover_constants.VOCAB_FEATURE_KEYS
_HASH_STRING_FEATURE_KEYS = cover_constants.HASH_STRING_FEATURE_KEYS
_LABEL_KEY = cover_constants.LABEL_KEY
_transformed_name = cover_constants.transformed_name

def preprocessing_fn(inputs):

    features_dict = {}

    ### START CODE HERE ###
    for feature in _SCALE_MINMAX_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using scaling of min_max function
        # Hint: Use tft.scale_by_min_max by passing in the respective column
        features_dict[_transformed_name(feature)] = tft.scale_by_min_max(data_col)

    for feature in _SCALE_01_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using scaling of 0 to 1 function
        # Hint: tft.scale_to_0_1
        features_dict[_transformed_name(feature)] = tft.scale_to_0_1(data_col)

    for feature in _SCALE_Z_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using scaling to z score
        # Hint: tft.scale_to_z_score
        features_dict[_transformed_name(feature)] = tft.scale_to_z_score(data_col)

    for feature in _VOCAB_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform using vocabulary available in column
        # Hint: Use tft.compute_and_apply_vocabulary
        features_dict[_transformed_name(feature)] = tft.compute_and_apply_vocabulary(data_col)

    for feature in _HASH_STRING_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transform by hashing strings into buckets
        # Hint: Use tft.hash_strings with the param hash_buckets set to 10
        features_dict[_transformed_name(feature)] = tft.hash_strings(data_col, hash_buckets=10)
    
    ### END CODE HERE ###  

    # No change in the label
    features_dict[_LABEL_KEY] = inputs[_LABEL_KEY]

    return features_dict

Overwriting cover_transform.py


In [29]:
transform = tfx.components.Transform(examples=example_gen.outputs['examples'],
                                    schema=imported_schema.outputs['schema'],
                                    preprocessing_fn='cover_transform.preprocessing_fn') # We are using the .py file that was created instead of the above function
context.run(transform)

Instructions for updating:
Use ref() instead.


2024-03-04 04:32:55.636709: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-04 04:32:55.636761: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-04 04:32:55.636781: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ece32b2d902d): /proc/driver/nvidia/version does not exist
2024-03-04 04:32:55.636982: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical o

INFO:tensorflow:Assets written to: ./pipeline/Transform/transform_graph/81/.temp_path/tftransform_tmp/5baac9aef942451baf2e4964ac54ce47/assets


INFO:tensorflow:Assets written to: ./pipeline/Transform/transform_graph/81/.temp_path/tftransform_tmp/5baac9aef942451baf2e4964ac54ce47/assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 81
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [30]:
transformed_examples_uri = transform.outputs['transformed_examples'].get()[0].uri
os.listdir(transformed_examples_uri)

['Split-eval', 'Split-train']

In [31]:
context.show(transform.outputs['transform_graph'])


In [32]:
uri_entrenamiento = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

archivos_tfrecord = [os.path.join(uri_entrenamiento, nombre)
                     for nombre in os.listdir(uri_entrenamiento)]

conjunto_transformado = tf.data.TFRecordDataset(archivos_tfrecord, compression_type="GZIP")

In [33]:
def ob_reg(conjunto, num_registros):
    
    registros = []
    
    for tfrecord in conjunto.take(num_registros):
        
        ejemplo_serializado = tfrecord.numpy()
        
        ejemplo = tf.train.Example()
        
        ejemplo.ParseFromString(ejemplo_serializado)
        
        diccionario_ejemplo = (MessageToDict(ejemplo))
        
        registros.append(diccionario_ejemplo)
        
    return registros

In [34]:
registros_muestra_xf = ob_reg(conjunto_transformado, 1)

pp.pprint(registros_muestra_xf)

[{'features': {'feature': {'Cover_Type': {'int64List': {'value': ['1']}},
                           'Elevation_xf': {'floatList': {'value': [0.11859784]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.9173228]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.9212598]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [0.21902902]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [0.047959913]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [-0.8517542]}},
                           'Slope_xf': {'floatList': {'value': [-0.94944674]}},
                           'Soil_Type_xf': {'int64List': {'value': ['6']}},
                           'Vertical_Distance_To_Hydrology_xf': {'floatList': {'value': [0.23197903]}},
                           'Wilderness_Area_xf': {'int64List': {'value': ['1']}}}}}]


### Metadatos

In [35]:
_util = 'util.py'

In [36]:
%%writefile {_util}

import tensorflow as tf
import pandas as pd
from google.protobuf.json_format import MessageToDict

def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

def display_types(types):
    # Helper function to render dataframes for the artifact and execution types
    table = {'id': [], 'name': []}
    for a_type in types:
        table['id'].append(a_type.id)
        table['name'].append(a_type.name)
    return pd.DataFrame(data=table)

def display_artifacts(store, artifacts, base_dir):
    # Helper function to render dataframes for the input artifacts
    table = {'artifact id': [], 'type': [], 'uri': []}
    for a in artifacts:
        table['artifact id'].append(a.id)
        artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
        table['type'].append(artifact_type.name)
        table['uri'].append(a.uri.replace(base_dir, './'))
    return pd.DataFrame(data=table)

def display_properties(store, node):
    # Helper function to render dataframes for artifact and execution properties
    table = {'property': [], 'value': []}
    
    for k, v in node.properties.items():
        table['property'].append(k)
        table['value'].append(
            v.string_value if v.HasField('string_value') else v.int_value)
    
    for k, v in node.custom_properties.items():
        table['property'].append(k)
        table['value'].append(
            v.string_value if v.HasField('string_value') else v.int_value)
    
    return pd.DataFrame(data=table)

Overwriting util.py


In [37]:
import util as u_t
import ml_metadata as mlmd
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = 'pipeline/metadata.sqlite'
connection_config.sqlite.connection_mode = 3
store = metadata_store.MetadataStore(connection_config)

### Acceso a artefactos almacenados

In [38]:
artifact_types = store.get_artifact_types()

In [39]:
u_t.display_types(artifact_types)

,id,name
0,14,Examples
1,17,ExampleStatistics
2,19,Schema
3,22,ExampleAnomalies
4,24,TransformGraph
5,25,TransformCache


In [40]:
list_artifact_type_schema = store.get_artifacts_by_type('Schema')
list_artifact_type_example = store.get_artifacts_by_type('Examples')
list_artifact_type_transform = store.get_artifacts_by_type('TransformGraph')
list_artifact_type_example_statistic = store.get_artifacts_by_type('ExampleStatistics')

In [41]:
u_t.display_artifacts(store, list_artifact_type_schema, '.')


,artifact id,type,uri
0,9,Schema,.//pipeline/SchemaGen/schema/9
1,12,Schema,.//pipeline/SchemaGen/schema/12
2,15,Schema,.//pipeline/SchemaGen/schema/15
3,18,Schema,.//pipeline/SchemaGen/schema/18
4,22,Schema,.//pipeline/SchemaGen/schema/22
5,25,Schema,.//pipeline/SchemaGen/schema/25
6,28,Schema,.//pipeline/SchemaGen/schema/28
7,31,Schema,.//pipeline/SchemaGen/schema/31
8,34,Schema,.//pipeline/SchemaGen/schema/34
9,38,Schema,.//pipeline/SchemaGen/schema/38


In [42]:
u_t.display_artifacts(store, list_artifact_type_example, '.')


,artifact id,type,uri
0,1,Examples,.//pipeline/CsvExampleGen/examples/1
1,2,Examples,.//pipeline/CsvExampleGen/examples/2
2,3,Examples,.//pipeline/CsvExampleGen/examples/3
3,4,Examples,.//pipeline/CsvExampleGen/examples/4
4,5,Examples,.//pipeline/ImportExampleGen/examples/5
5,6,Examples,.//pipeline/ImportExampleGen/examples/6
6,7,Examples,.//pipeline/CsvExampleGen/examples/7
7,10,Examples,.//pipeline/CsvExampleGen/examples/10
8,13,Examples,.//pipeline/CsvExampleGen/examples/13
9,16,Examples,.//pipeline/CsvExampleGen/examples/16


### Seguimiento de Artefactos

In [45]:
def get_parent_artifacts(store, artifact):

    ### START CODE HERE ###
    
    # Get the artifact id of the input artifact
    artifact_id = artifact.id
    
    # Get events associated with the artifact id
    artifact_id_events = store.get_events_by_artifact_ids([artifact_id])
    
    # From the `artifact_id_events`, get the execution ids of OUTPUT events.
    # Cast to a set to remove duplicates if any.
    execution_id = set( 
        event.execution_id
        for event in artifact_id_events # @REPLACE
        if event.type == metadata_store_pb2.Event.OUTPUT # @REPLACE
    )
    
    # Get the events associated with the execution_id
    execution_id_events = store.get_events_by_execution_ids(execution_id)

    # From execution_id_events, get the artifact ids of INPUT events.
    # Cast to a set to remove duplicates if any.
    parent_artifact_ids = set( 
        event.artifact_id
        for event in execution_id_events
        if event.type == metadata_store_pb2.Event.INPUT
    )
    
    # Get the list of artifacts associated with the parent_artifact_ids
    parent_artifact_list = [artifact for artifact in store.get_artifacts_by_id(parent_artifact_ids)]

    ### END CODE HERE ###
    
    return parent_artifact_list

In [48]:
# Get an artifact instance from the metadata store
artifact_instance = store.get_artifacts_by_type('TransformGraph')[0]

# Retrieve the parent artifacts of the instance
parent_artifacts = get_parent_artifacts(store, artifact_instance)

# Display the results
u_t.display_artifacts(store, parent_artifacts, '.')

,artifact id,type,uri
0,59,Examples,.//pipeline/CsvExampleGen/examples/59
1,62,Schema,.//pipeline/ImportSchemaGen/schema/62
